In [10]:
class A:
    def __init__(self):
        pass

class B:
    def __init__(self):
        pass
    
    def __getattr__(self, name):
        print("{} no attr {}".format(self, name))
        

In [4]:
a = A()
a.aaa()

AttributeError: 'A' object has no attribute 'aaa'

In [12]:
b = B()
b.bbb()

<__main__.B object at 0x000002612239A1D0> no attr bbb


TypeError: 'NoneType' object is not callable

In [16]:
b = B()
b.name

<__main__.B object at 0x00000261223A6A20> no attr name


# 问题

- 你想将某个实例的属性访问代理到内部另一个实例中去，目的可能是作为继承的
一个替代方法或者实现代理模式。

## 解决方案

- 简单来说，代理是一种编程模式，它将某个操作转移给另外一个对象来实现。最简
单的形式可能是像下面这样：

In [27]:
class A:
    def spam(self, x):
        print("A.spam.")
        pass
    
    def foo(self):
        print("A.foo")
        pass
    
class B1:
    def __init__(self):
        self._a = A()
        
    def spam(self, x):
        return self._a.spam(x)
        
    def foo(self):
        return self._a.foo()
    
    def bar(self):
        pass

In [28]:
b = B1()
b.spam(1)
b.foo()

A.spam.
A.foo


- 如果仅仅就两个方法需要代理，那么像这样写就足够了。但是，如果有大量的方法
需要代理，那么使用`__getattr__()` 方法或许或更好些：

In [25]:
class B2:
    def __init__(self):
        self._a = A()
        
    def bar():
        pass
    
    def __getattr__(self, name):
        print('getattr:{}'.format(name))
        return getattr(self._a, name)
    

In [31]:
b = B2()

b.foo()
b.spam(222)

getattr:foo
A.foo
getattr:spam
A.spam.


- `__getattr__` 方法是在访问attribute 不存在的时候被调用，使用演示：

- 另外一个代理例子是实现代理模式，例如：

In [32]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
        
    def __getattr__(self, name):
        print("getattr:{}".format(name))
        return getattr(self._obj, name)
    
    def __setattr__(self, name, value):
        if name.startswith("_"):
            super().__setattr__(name, value)
        else:
            print("setattr:{}--{}".format(name, value))
            setattr(self._obj, name, value)
            
    def __delattr__(self, name):
        if name.startswith("_"):
            super().__delattr__(name)
        else:
            print("delattr:{}".format(name))
            delattr(self._obj, name)

- 使用这个代理类时，你只需要用它来包装下其他类即可：

In [35]:
class Spam:
    def __init__(self, x):
        self.x = x
        
    def bar(self, y):
        print("Spam.bar", self.x, y)
        
s = Spam(2)
p = Proxy(s)

print(p.x)

p.bar(30)
p.x = 40

getattr:x
2
getattr:bar
Spam.bar 2 30
setattr:x--40


- 通过自定义属性访问方法，你可以用不同方式自定义代理类行为(比如加入日志功
能、只读访问等)。

## 讨论

- 代理类有时候可以作为继承的替代方案。例如，一个简单的继承如下：

In [42]:
class A:
    def spam(self, x):
        print("A.spam", x)
        
    def foo(self):
        print("A.foo")
        
class B(A):
    def spam(self, x):
        print("B.spam", x)
        super().spam(x)
        
    def foo(self):
        print("B.foo")
        super().foo()

In [43]:
b = B()
b.spam(1)
b.foo()

B.spam 1
A.spam 1
B.foo
A.foo


- 使用代理的话，就是下面这样：

In [44]:
class A:
    def spam(self, x):
        print("A.spam", x)
    def foo(self):
        print("A.spam")
        
class B:
    def __init__(self):
        self._a = A()
        
    def __getattr__(self, name):
        return getattr(self._a, name)

In [47]:
b = B()

b.spam(1)
b.foo()

A.spam 1
A.spam


- 当实现代理模式时，还有些细节需要注意。首先，`__getattr__()` 实际是一个后备
方法，只有在属性不存在时才会调用。因此，如果代理类实例本身有这个属性的话，那
么不会触发这个方法的。另外，`__setattr__()` 和`__delattr__()` 需要额外的魔法来
区分代理实例和被代理实例`_obj` 的属性。一个通常的约定是只代理那些不以`下划线_`
开头的属性(代理类只暴露被代理类的公共属性)。

- 还有一点需要注意的是，`__getattr__()` 对于大部分以双下划线`(__)` 开始和结尾
的属性并不适用。比如，考虑如下的类：